## **PROYECTO PILOTO AUTONOMA DESERCION**

Luego de revisar varias veces la data, se decide empezar por la tabla con mayor detalle que es EstudianteAsignatura, que tiene información no solo de las asignaturas que el estudiante ha matriculado en un periodo, sino también qué grupos ha tomado de dicha asigantura e incluso qué docentes dictaron clase en cada grupo.

La tabla base del proyecto es EstudianteSemestre, que es la que indica si el estudiante tomó asignaturas en ese semestre o no.   Llegaremos posteriormente a ella.

Usaremos solo una conexión a la base de datos.  Solo ejecutar el siguiente codigo una vez.

Para queries solo se requiere lo siguiente y el resultado queda en el dataframe
1. query = 'SELECT ...'
2. df = pd.read_sql(query, cnx)

Para DML se requiere:
1. crear un cursor = cnx.cursor()
2. query = 'update...'
3. cursor.execute(query)
4. cnx.commit()
5. cursor.close()

In [2]:
from sqlalchemy import create_engine
import pandas as pd

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
conexionMySQL = create_engine(cadena_conexion)


# OJO CAMBIOS EN EL MODELO DE DATOS HECHOS DIRECTAMENTE EN MYSQL
""" 
1.  se cambió EstudiantesMatriculas(CODIGO) -> EstudiantesMatriculas(ID_ESTUDIANTE)
2.  se creó la tabla PERIODOS para que se incluyera una nueva columna ORDEN que lleva el orden de los periodos regulares
"""

' \n1.  se cambió EstudiantesMatriculas(CODIGO) -> EstudiantesMatriculas(ID_ESTUDIANTE)\n2.  se creó la tabla PERIODOS para que se incluyera una nueva columna ORDEN que lleva el orden de los periodos regulares\n'

ALGORITMO e HIPOTESIS

1. creamos PERIODOS a partir de los distintos períodos en AXM y que nos queden ordenados, y en una lista
pues luego vamos a confirmar si luego de dos períodos estos estudiantes han dejado de matricular.
1. creamos DATOS a partir de Matrícula.
2. adicionamos la columna LABEL que debe ser 1 si luego de 2 periodos,  el estudiante dejó de matricular mínimo 2 periodos consecutivos o más y no fue que se graduó.  De lo contrario debe ser 0.

BASE:  puede ser que nos paramos al final de un semestre, o al prinpcio de un semestre.    Este segundo caso nos permitiria jugar con 202302.  Asi que escogemos asumir que estamos iniciando un semestre y por tanto no tenemos notas del semestre actual.

HIPOTESIS

1.  promedio acumulado de 4 semestres anteriores
2.  promedio por semestre de 4 semestres anteriores
3.  cantidad de asignaturas que tomó el semestre actual y 4 semestres hacia atras
4.  cantidad de créditos que tomó semestre actual y 4 semestres anteriores
5.  cantidad de créditos que perdió 4 semestres anteriores
6.  cantidad de asignaturas que perdió 4 semestres anteriores
7.  si asistió a actividades extracurriculares 4 semestres anteriores
8.  que tan heterogenoes fueron los grupos que toma este semestre y 4 anteriores (desvStd edad, relación F:M)
9.  tuvo mora en 4 semestres anteriores
10. docentes - edad promedio docentes este semestre y anteriores
11. asignaturas que tomó y sus notas en 4 semestres anteriores (son un jurgo de variables)


Algoritmo Paso 1:  creamos PERIODOS

In [3]:
# Traemos los PERIODOS
query = """
SELECT distinct PERIODO 
FROM EstudianteAsignatura
ORDER BY PERIODO"""

# Leer resultados en un DataFrame y lo convertimos en una lsita
PERIODOS = pd.read_sql_query(query, conexionMySQL)
print(PERIODOS)

# 2022 10 09 NOTA IMPORTANTE:
# En la tabla EstudianteAsignatura NO APARECE ninguna fila que indique algún estudainte hizo cursos de verano
# De hecho, todas las filas tienen EstudianteAsignatura.CURSO_VERANO   = 'NO'.
# Sin embargo, en EstudianteMatricula, SI APARECEN filas como si algún estudiante hubiera hecho un curso de verano
# Preguntar a ERIK.

# En este escenario, todos los períodos son REGULARES, pero puede que no sea así en otro caso.

    PERIODO
0    201801
1    201803
2    201901
3    201903
4    202001
5    202003
6    202101
7    202103
8    202201
9    202203
10   202301
11   202303


In [4]:
# Treamos una lista de estudiantes
query = """
SELECT  id_estudiante, MIN(periodo)
FROM EstudianteAsignatura
GROUP BY id_estudiante
ORDER BY id_estudiante"""

# Leer resultados en un DataFrame y lo convertimos en una lsita
primeraMATRICULA = pd.read_sql_query(query, conexionMySQL)

Algoritmo Paso 2:  cremos DATOS a partir EstudianteAsignatura(id_estudiante, periodo)

In [5]:
# Hacer una consulta SQL
query = """
SELECT DISTINCT ea.id_estudiante, ea.periodo 
FROM EstudianteAsignatura ea
LEFT JOIN Estudiante e

"""

# Leer resultados en un DataFrame
DATOS = pd.read_sql_query(query, conexionMySQL)
DATOS.shape

(832, 2)

Ahora, a datos le incluimos la fecha de admisión, y la de egresado para confirmar si es desertor o no.
ERIK me confirma que la validación solo se hace tomando en cuenta PERIODOS REGULARES.

Algoritmo paso 3:  crear columna LABEL y que sea 1 si dado una flia en matricula, NO hay fila en matrícula durante 2 periodos consecutivos posteriores.

In [4]:
DATOS['LABEL'] = 0

for e in range(0, len(lEstudiantes)):

    # print('Estudiante:', lEstudiantes[e])

    for p in range(0, len(lPeriodos)-2):  # los ultimos 2 períodos no los analizamos

        if ((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p])).any():

            # si ocurre un error es porque los indices se salieron de madre
            try:
                semestre_1 = ((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p+1])).any() 
                semestre_2 = ((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p+2])).any() 
            except IndexError:
               semestre_1 = False
               semestre_2 = False

            if semestre_1 & semestre_2:    
                # print(      'si matriculó ambos periodos')           
                pass 
            else:
                # print(      'no matriculó ambos períodos')
                MATRICULAS.loc[((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p])),['LABEL']] = 1
                break

Hay que revisar de los registros que quedaron marcados como desertores (LABEL==1), que no ocurra que fue que el estudiante se graduó.

TAREA
Tomamos esos registros, les adicionamos la fecha de grado si la tienen, convertida a formato YYYYNN, donde NN = 01 o 03 si se graduó en el primer o segundo semestre
y miramos que el PERIODO P que aparece como desertor no ven a ser que en p+1 o p+2 se graduó...  





In [5]:
MATRICULAS.dtypes

id_estudiante    int64
periodo          int64
LABEL            int64
dtype: object

In [6]:
import numpy as np
# hacer outerjoin entre MATRICULAS y GRADUADOS para meter la 
# fecha de grado
# luego hacer un for con esos registros que tienen fecha de grado no nula
# y convertirla al formato YYYYNN donde NN=01 o 03 según el grado haya ocurrido
# en alguno de esos semestres.

# si periodo_grado < periodo +2 quiere decir que se graduó ANTES de haber 
# pasado dos semestres y por tanto no puede mardarse como desertor.

query = """
SELECT id_estudiante, fecha_grado 
FROM estudiante
WHERE fecha_grado is not null"""

GRADUADOS = pd.read_sql_query(query, conexionMySQL)

# creamos periodo_grado en el semestre en que este ocurrió (01 o 03)
GRADUADOS['periodo_grado'] = np.where(GRADUADOS['fecha_grado'].dt.month <= 6, 
                   GRADUADOS['fecha_grado'].dt.year.astype(str) + '01',
                   GRADUADOS['fecha_grado'].dt.year.astype(str) + '03')


MATRICULAS = pd.merge(MATRICULAS, GRADUADOS, on=['id_estudiante'], how='left')
MATRICULAS['periodo_grado'] = MATRICULAS['periodo_grado'].astype('Int64')

In [7]:
import numpy as np

def encontrar_indice(L, x):
  for i, elemento in enumerate(L):
    if elemento == x:
      return i
  return -1


# Recorremos matriculas
for index, m in MATRICULAS.iterrows():
    
    # Entoncramos las filas donde están marcados como desertores
    valor = str(m['periodo_grado'])

    # si está marcado como desertor y el período de grado NO ES NULO
    if (m['LABEL'] == 1 and valor != '<NA>'):

      # miramos si se graduó el semestre i, i + 1 o en el i + 2. si eso ocurrió
      # NO ES DESERTOR.
    
      i = encontrar_indice(lPeriodos, m['periodo'])
      j = encontrar_indice(lPeriodos, m['periodo_grado'])

      if j== i or j == i+1 or j==i+2:

        print('caso encontrado:', m)
        m['LABEL'] == 0

caso encontrado: id_estudiante                2185298
periodo                       202103
LABEL                              1
fecha_grado      2022-11-04 00:00:00
periodo_grado                 202203
Name: 140, dtype: object


En este punto ya tenemos marcados los desertores (dos períkodos consecutivos sin matrícula) y además revisando si fue que se graduó en el período i, i+1 o i+2.


Hipotesis 1:  incluir 4 promedios semestre anteriores.

In [31]:
MATRICULAS.dtypes

id_estudiante             int64
periodo                   int64
LABEL                     int64
fecha_grado      datetime64[ns]
periodo_grado             Int64
dtype: object

In [38]:
MATRICULAS.head()

,id_estudiante,periodo,LABEL,fecha_grado,periodo_grado
0,2175510,201801,1,NaT,<NA>
1,2235429,202303,0,NaT,<NA>
2,2222049,202203,0,NaT,<NA>
3,2222049,202301,0,NaT,<NA>
4,2222049,202201,0,NaT,<NA>


In [32]:
query = """
SELECT distinct codigo as id_estudiante, CAST(periodo as SIGNED) as periodo, prom_ponderado_acu, prom_ponderado_sem
FROM estudiantesmatriculas
WHERE activo_acad = 'S' 
AND activo_fin = 'S'
"""

promedios = pd.read_sql_query(query, conexionMySQL)

In [37]:
promedios.head()

,id_estudiante,periodo,prom_ponderado_acu,prom_ponderado_sem
0,2206167,202103,4.7,NaN
1,2210056,202103,4.6,NaN
2,2130302,201901,3.8,3.9
3,2151229,201901,4.5,4.6
4,2157111,201901,4.4,4.5


In [34]:
promedios.dtypes

id_estudiante           int64
periodo                 int64
prom_ponderado_acu    float64
prom_ponderado_sem    float64
dtype: object

In [35]:
# hacemos join entre MATRICULAS y PROMEDIOS por el id_estudiante, y traemos de promedios todos los periodos que sean igual o menores al periodo en MATRICULAS (4 filas)
temp = MATRICULAS.merge(promedios, on=['id_estudiante'], how='left' )
temp = temp.loc[temp['periodo_x'] >= temp['periodo_y'], ['id_estudiante','periodo_x','periodo_y','prom_ponderado_acu','prom_ponderado_sem']]
temp = temp.sort_values(by=['id_estudiante', 'periodo_x', 'periodo_y'], ascending=[True, False, False])




In [36]:
temp

,id_estudiante,periodo_x,periodo_y,prom_ponderado_acu,prom_ponderado_sem
1309,2136689,201803,201803.0,3.3,0.2
1994,2146669,201801,201801.0,4.6,4.6
1287,2146707,201801,201801.0,4.2,4.3
2035,2146709,201801,201801.0,4.0,3.8
1993,2146710,201801,201801.0,4.2,4.4
...,...,...,...,...,...
848,2216297,202303,202103.0,4.5,NaN
850,2216297,202301,202103.0,4.5,NaN
847,2216297,202203,202103.0,4.5,NaN
851,2216297,202201,202103.0,4.5,NaN


In [9]:
# Recorremos matriculas
df = estudiantesMatriculas
df_pivot = df.pivot(index='id_estudiante', columns='periodo', values='prom_ponderado_acu')
df_pivot

periodo,201801,201803,201901,201903,202001,202003,202101,202103,202104
id_estudiante,,,,,,,,,
2130302,NaN,3.8,3.8,3.8,3.9,3.9,3.9,3.9,3.9
2136170,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
2136689,NaN,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2140149,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2141083,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2216141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,3.5
2216142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.2,4.2
2216143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.1,4.1
